In [ ]:
%sql
-- Create the gold fact table (run once or if schema changes needed)
CREATE TABLE IF NOT EXISTS gold.ipl_venues_stats (
  venue_sk BIGINT,
  venue_name STRING,
  season INT,

  team1_sk BIGINT,
  team2_sk BIGINT,

  toss_winner_team_sk BIGINT,
  toss_decision STRING,          -- bat / field

  match_winner_team_sk BIGINT,   -- winner team

  total_matches INT,
  team1_wins INT,
  team2_wins INT,

  batting_first_wins INT,
  chasing_wins INT,

  avg_first_innings_score DOUBLE,
  avg_second_innings_score DOUBLE,

  ingestion_ts TIMESTAMP
)
USING DELTA
TBLPROPERTIES (
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
);
-- Optional: PARTITIONED BY (season) for better performance with many seasons

In [ ]:
from pyspark.sql import functions as F

# Read silver tables
fact_matches = spark.table("silver.fact_matches")
fact_deliveries = spark.table("silver.fact_deliveries")
dim_venue = spark.table("silver.dim_venue")

In [ ]:
# Calculate main innings totals (exclude super overs)
innings_df = fact_deliveries.filter(F.col("is_super_over") == 0) \
    .groupBy("match_id", "inning") \
    .agg(F.sum("total_runs").alias("innings_runs_raw"))

first_innings = innings_df.filter(F.col("inning") == 1) \
    .select("match_id", F.col("innings_runs_raw").cast("double").alias("first_innings_score"))

second_innings = innings_df.filter(F.col("inning") == 2) \
    .select("match_id", F.col("innings_runs_raw").cast("double").alias("second_innings_score"))

In [ ]:
# Base: match data + venue name (fixed join) + innings scores
base_df = fact_matches.select(
    "match_id",
    "season",
    "venue_sk",  # This comes from fact_matches (foreign key to dim_venue.venue_id)
    "team1_sk",
    "team2_sk",
    "toss_winner_sk",
    "toss_decision",
    "winner_sk"
).join(
    dim_venue.select(
        F.col("venue_id").alias("venue_sk"),  # Alias to match fact_matches.venue_sk
        F.col("official_name").alias("venue_name")
    ),
    "venue_sk"  # Joins fact_matches.venue_sk == dim_venue.venue_id (aliased)
).join(first_innings, "match_id", "left") \
 .join(second_innings, "match_id", "left")

In [ ]:
# Determine team that batted first
batting_first_team_sk = F.when(F.col("toss_decision") == "bat", F.col("toss_winner_sk")) \
    .otherwise(
        F.when(F.col("toss_winner_sk") == F.col("team1_sk"), F.col("team2_sk"))
        .otherwise(F.col("team1_sk"))
    ).alias("batting_first_team_sk")

In [ ]:
# Assemble gold DataFrame
gold_df = base_df \
    .withColumn("batting_first_team_sk", batting_first_team_sk) \
    .withColumn("toss_winner_team_sk", F.col("toss_winner_sk")) \
    .withColumn("match_winner_team_sk", F.col("winner_sk")) \
    .withColumn("total_matches", F.lit(1).cast("int")) \
    .withColumn("team1_wins", F.when(F.col("winner_sk") == F.col("team1_sk"), 1).otherwise(0).cast("int")) \
    .withColumn("team2_wins", F.when(F.col("winner_sk") == F.col("team2_sk"), 1).otherwise(0).cast("int")) \
    .withColumn("batting_first_wins", F.when(F.col("winner_sk") == F.col("batting_first_team_sk"), 1).otherwise(0).cast("int")) \
    .withColumn("chasing_wins", F.when((F.col("winner_sk").isNotNull()) & (F.col("winner_sk") != F.col("batting_first_team_sk")), 1).otherwise(0).cast("int")) \
    .withColumn("avg_first_innings_score", F.coalesce(F.col("first_innings_score"), F.lit(0.0))) \
    .withColumn("avg_second_innings_score", F.coalesce(F.col("second_innings_score"), F.lit(0.0))) \
    .select(
        "venue_sk",
        "venue_name",
        "season",
        "team1_sk",
        "team2_sk",
        "toss_winner_team_sk",
        "toss_decision",
        "match_winner_team_sk",
        "total_matches",
        "team1_wins",
        "team2_wins",
        "batting_first_wins",
        "chasing_wins",
        "avg_first_innings_score",
        "avg_second_innings_score",
        F.current_timestamp().alias("ingestion_ts")
    )

In [ ]:
# Write to gold table
gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.ipl_venues_stats")

In [ ]:
# Optimize
spark.sql("OPTIMIZE gold.ipl_venues_stats ZORDER BY (season, venue_sk)")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [ ]:
%sql
SELECT venue_sk, venue_name, season,
       SUM(total_matches) AS total_matches,
       SUM(batting_first_wins) AS batting_first_wins,
       SUM(chasing_wins) AS chasing_wins,
       AVG(avg_first_innings_score) AS true_avg_first_innings,
       AVG(avg_second_innings_score) AS true_avg_second_innings
FROM gold.ipl_venues_stats
GROUP BY venue_sk, venue_name, season